# Transformer Apply for QQQ ETF

## 数据预处理：

In [3]:
import sys
sys.path.append('/workspaces/AI-Trader')

import torch
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from models.ml.transformer import TimeSeriesTransformer  # Import the optimized model
from features.feature_store import process_features  # Import feature processing pipeline
# Data loading and preprocessing
def load_and_preprocess_data(file_path, seq_len=30):
    """
    Load and preprocess time series data for model training.

    Args:
        file_path (str): Path to the CSV file.
        seq_len (int): Sequence length for time series.

    Returns:
        X (torch.Tensor): Input sequences.
        y (torch.Tensor): Target values.
        scaler (MinMaxScaler): Scaler for inverse transformation.
    """
    # Read data
    data = pd.read_csv(file_path)
    
    # Select needed columns
    data = data[['Date', 'Close']]
    
    # Convert date format and sort (address pandas warning)
    data['Date'] = pd.to_datetime(data['Date'], utc=True)  # Set utc=True to address warning
    data = data.sort_values('Date')

    # Process features using the feature pipeline
    processed_features = process_features(data)
    
    # Normalize the 'Close' column for sequence creation
    scaler = MinMaxScaler(feature_range=(0, 1))
    processed_features['Close'] = scaler.fit_transform(processed_features[['Close']])
    
    # Create sequences efficiently
    close_values = processed_features['Close'].values
    X, y = create_sequences(close_values, seq_len)
    
    return X, y, scaler

def create_sequences(data, seq_len):
    """
    Create time series sequences efficiently.

    Args:
        data (np.ndarray): Input data (e.g., features).
        seq_len (int): Sequence length.

    Returns:
        X (torch.Tensor): Input sequences.
        y (torch.Tensor): Target values.
    """
    sequences = []
    targets = []
    
    for i in range(len(data) - seq_len):
        seq = data[i:i + seq_len]
        target = data[i + seq_len]
        sequences.append(seq)
        targets.append(target)
    
    # Convert to numpy arrays first, then to tensors
    sequences_np = np.array(sequences)
    targets_np = np.array(targets)
    
    return torch.from_numpy(sequences_np).float(), torch.from_numpy(targets_np).float()


In [4]:
# Data splitting
def split_data(X, y, train_ratio=0.7, val_ratio=0.15):
    """Split data into training, validation, and test sets"""
    train_size = int(train_ratio * len(X))
    val_size = int(val_ratio * len(X))
    
    # Split data
    X_train, X_val = X[:train_size], X[train_size:train_size + val_size]
    X_test = X[train_size + val_size:]
    y_train, y_val = y[:train_size], y[train_size:train_size + val_size]
    y_test = y[train_size + val_size:]
    
    return X_train, X_val, X_test, y_train, y_val, y_test


## 模型训练

In [5]:
# Model training function
def train_model(model, X_train, y_train, X_val, y_val, num_epochs=50, batch_size=32, 
               patience=10, lr=0.001, weight_decay=1e-5):
    """Train the model with early stopping and learning rate scheduler"""
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    
    # Move data to device (properly)
    X_train, y_train = X_train.to(device), y_train.to(device)
    X_val, y_val = X_val.to(device), y_val.to(device)
    
    # Initialize optimizer and scheduler
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=0.5, patience=5, verbose=True
    )
    
    # Training tracking
    train_losses, val_losses = [], []
    train_accuracies, val_accuracies = [], []
    best_val_loss = float('inf')
    patience_counter = 0
    
    print("Starting model training...")
    for epoch in range(num_epochs):
        # Training phase
        train_loss, train_acc = model.train_epoch(X_train, y_train, optimizer, batch_size=batch_size)
        
        # Validation phase
        val_loss, val_acc = model.validate(X_val, y_val, batch_size=batch_size)
        
        # Learning rate scheduling
        scheduler.step(val_loss)
        
        # Record metrics
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        train_accuracies.append(train_acc)
        val_accuracies.append(val_acc)
        
        # Print progress
        print(f"Epoch [{epoch + 1}/{num_epochs}], "
              f"Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, "
              f"Train Acc: {train_acc:.2f}, Val Acc: {val_acc:.2f}, "
              f"LR: {optimizer.param_groups[0]['lr']:.6f}")
        
        # Early stopping check
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
            # Save best model
            torch.save(model.state_dict(), 'best_transformer_model.pth')
            print(f"Model saved at epoch {epoch + 1} with validation loss: {val_loss:.4f}")
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"Early stopping triggered after {epoch + 1} epochs")
                break
    
    return model, train_losses, val_losses, train_accuracies, val_accuracies





In [6]:
# Evaluation and visualization
def evaluate_model(model, X_test, y_test, batch_size=32):
    """Evaluate model on test data and visualize results"""
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    X_test, y_test = X_test.to(device), y_test.to(device)
    
    # Load best model
    model.load_state_dict(torch.load('best_transformer_model.pth'))
    model.eval()
    
    # Test evaluation
    print("Evaluating model on test set...")
    test_loss, test_acc = model.validate(X_test, y_test, batch_size=batch_size)
    print(f"Test Loss: {test_loss:.4f}, Direction Accuracy: {test_acc:.2f}")
    
    # Prediction and metrics
    predictions = model.predict(X_test)
    y_test_np = y_test.cpu().numpy()
    
    mse = mean_squared_error(y_test_np, predictions)
    mae = mean_absolute_error(y_test_np, predictions)
    direction_accuracy = np.mean(np.sign(predictions) == np.sign(y_test_np))
    
    print(f"Detailed evaluation metrics:")
    print(f"MSE: {mse:.4f}")
    print(f"MAE: {mae:.4f}")
    print(f"Direction Accuracy: {direction_accuracy:.2f}")
    
    # Visualize results
    visualize_results(train_losses, val_losses, train_accuracies, val_accuracies, 
                     y_test_np, predictions)
    
    return mse, mae, direction_accuracy

In [7]:
def visualize_results(train_losses, val_losses, train_accuracies, val_accuracies, 
                     y_test, predictions):
    """Visualize training history and predictions"""
    # Plot loss curves
    plt.figure(figsize=(16, 8))
    plt.subplot(1, 2, 1)
    plt.plot(train_losses, label='Train Loss')
    plt.plot(val_losses, label='Val Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.yscale('log')  # Use log scale for better loss visualization
    plt.grid(True)
    plt.legend()
    plt.title('Loss Curve')
    
    # Plot accuracy curves
    plt.subplot(1, 2, 2)
    plt.plot(train_accuracies, label='Train Accuracy')
    plt.plot(val_accuracies, label='Val Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.grid(True)
    plt.legend()
    plt.title('Direction Accuracy Curve')
    plt.tight_layout()
    plt.savefig('transformer_training_curves.png')
    plt.show()
    
    # Plot predictions vs actual values
    plt.figure(figsize=(14, 7))
    plt.plot(y_test, label='Actual Values', color='blue')
    plt.plot(predictions, label='Predicted Values', color='red', linestyle='--')
    plt.title('Prediction Results Comparison')
    plt.xlabel('Time Step')
    plt.ylabel('Value')
    plt.legend()
    plt.grid(True)
    plt.savefig('prediction_results.png')
    plt.show()


In [8]:
# Main execution
if __name__ == "__main__":
   # Load and preprocess data
    file_path = '/workspaces/AI-Trader/data/raw/qqq_history.csv'
    seq_len = 30
    X, y, scaler = load_and_preprocess_data(file_path, seq_len)

    # Get feature dimension dynamically
    feature_dim = X.shape[2]  # Number of features

    # Split data
    X_train, X_val, X_test, y_train, y_val, y_test = split_data(X, y)
    
    # Model hyperparameters
    feature_dim = 1  # Single feature (closing price)
    num_heads = 4  # Number of attention heads
    hidden_dim = 128  # Hidden dimension
    num_layers = 2  # Number of Transformer layers
    dropout = 0.2  # Dropout rate
    
    # Initialize model
    model = TimeSeriesTransformer(
        feature_dim=feature_dim,
        seq_len=seq_len,
        num_heads=num_heads,
        hidden_dim=hidden_dim,
        num_layers=num_layers,
        dropout=dropout
    )
    
    # Train model
    model, train_losses, val_losses, train_accuracies, val_accuracies = train_model(
        model, X_train, y_train, X_val, y_val
    )
    
    # Evaluate model
    evaluate_model(model, X_test, y_test)

IndexError: tuple index out of range

## 预测与策略生成

In [ ]:

# 预测
predictions = model.predict(X_test)

# 反归一化
predictions = scaler.inverse_transform(predictions.detach().numpy())
y_test = scaler.inverse_transform(y_test.numpy().reshape(-1, 1))

AttributeError: 'numpy.ndarray' object has no attribute 'detach'

In [ ]:
# 生成交易信号：根据预测值生成买入、卖出或持有信号：
import numpy as np

# 简单策略：如果预测价格高于当前价格，则买入；否则卖出
signals = []
for i in range(len(predictions)):
    if predictions[i] > y_test[i]:
        signals.append('Buy')
    else:
        signals.append('Sell')

# 将信号与日期对应
results = pd.DataFrame({
    'Date': data['Date'].iloc[train_size + seq_len:].values,
    'Actual': y_test.flatten(),
    'Predicted': predictions.flatten(),
    'Signal': signals
})
print(results.head())

                        Date      Actual   Predicted Signal
0  2020-01-27 00:00:00-05:00  211.204575 -288.426941   Sell
1  2020-01-28 00:00:00-05:00  214.448685 -288.426941   Sell
2  2020-01-29 00:00:00-05:00  214.797256 -288.426941   Sell
3  2020-01-30 00:00:00-05:00  215.562286 -288.426941   Sell
4  2020-01-31 00:00:00-05:00  212.143906 -288.426941   Sell


## 评估策略

In [ ]:
from sklearn.metrics import mean_squared_error

mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)
print(f'MSE: {mse:.4f}, RMSE: {rmse:.4f}')

MSE: 413447.2500, RMSE: 642.9986
